# Memory

In [102]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        super().__init__()
        self.messages = []

    def add_messages(self, messages):
        self.messages.extend(messages)

    def clear(self):
        self.messages = []

    def __repr__(self):
        return str(self.messages)

In [104]:
# item(key=session_id, value=InMemoryHistory인스턴스)
store = {}

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [105]:
from langchain_core.messages import HumanMessage

history = get_by_session_id('user1')
# history.add_messages(HumanMessage(content='점심 뭐 먹지?'))
print(store)

{'user1': []}


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

"""
prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘하는 AI 어시스턴트야'),
    MessagesPlaceholder(variable_name='history'),
    ('human', '{query}')
])
"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template('너는 {skill}을 잘하는 AI 어시스턴트야'),
    MessagesPlaceholder(variable_name='history'),
    HumanMessagePromptTemplate.from_template('{query}')
])
model = ChatOpenAI(model_name='gpt-4o-mini')

chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'  # MessagesPlaceholder의 variable_name과 맞춰줘야 함
)

In [107]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '다람쥐는 도토리 나무를 세 그루 키우고 있다'},
    config={'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='다람쥐가 도토리 나무를 세 그루 키우고 있다니, 흥미로운 이야기네요! 다람쥐들은 도토리를 좋아하고, 때로는 도토리를 땅에 묻어 두었다가 나중에 먹곤 하죠. 혹시 다람쥐와 도토리 나무에 대한 이야기를 더 공유해 주실 건가요? 또는 어떤 특정한 질문이 있으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 44, 'total_tokens': 144, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CBwc7WYTU5R5fOvGYKt7MtgSdLCdV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--c42da767-5196-4d0c-a98a-77e361c9ceb2-0' usage_metadata={'input_tokens': 44, 'output_tokens': 100, 'total_tokens': 144, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [108]:
store

{'user1': [],
 'squirrel': [HumanMessage(content='다람쥐는 도토리 나무를 세 그루 키우고 있다', additional_kwargs={}, response_metadata={}), AIMessage(content='다람쥐가 도토리 나무를 세 그루 키우고 있다니, 흥미로운 이야기네요! 다람쥐들은 도토리를 좋아하고, 때로는 도토리를 땅에 묻어 두었다가 나중에 먹곤 하죠. 혹시 다람쥐와 도토리 나무에 대한 이야기를 더 공유해 주실 건가요? 또는 어떤 특정한 질문이 있으신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 44, 'total_tokens': 144, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CBwc7WYTU5R5fOvGYKt7MtgSdLCdV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c42da767-5196-4d0c-a98a-77e361c9ceb2-0', usage_metadata={'input_tokens': 44, 'output_tokens': 100, 'total_tokens': 144, 'input_token_details': {'audio': 0, 'c

In [109]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼는 당근 농장을 다섯 개나 운영하고 있다'},
    config={'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='토끼가 당근 농장을 다섯 개나 운영하고 있다니, 정말 귀엽고 창의적인 설정이네요! 토끼들은 당근을 아주 좋아하니까, 각 농장을 어떻게 관리하는지도 궁금해집니다. 토끼 친구가 어떤 방식으로 농사를 짓고 있는지, 아니면 이 이야기에 대해 더 많은 내용을 추가하고 싶으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 167, 'total_tokens': 252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CBwc91rD4NvqkPUQFLoW0LLrnsAmJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--00067887-10ef-404d-a230-035ce0dc6ca3-0' usage_metadata={'input_tokens': 167, 'output_tokens': 85, 'total_tokens': 252, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [110]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '다람쥐는 도토리 나무를 몇 그루 가지고 있지?'},
    config={'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='다람쥐는 도토리 나무를 세 그루 가지고 있다고 했죠! 이 도토리 나무들은 다람쥐에게 먹이를 제공하고, 그늘을 만들어 줄 수 있을 것 같아요. 혹시 다람쥐와 도토리 나무에 대해 더 이야기해볼까요? 아니면 다른 동물이나 이야기도 나눌까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 279, 'total_tokens': 362, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CBwcBl7L3YVC1vxS8bq3unSRGT27l', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--20c81662-a5c6-4724-a07e-de57a41effa4-0' usage_metadata={'input_tokens': 279, 'output_tokens': 83, 'total_tokens': 362, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [112]:
store['squirrel']

[HumanMessage(content='다람쥐는 도토리 나무를 세 그루 키우고 있다', additional_kwargs={}, response_metadata={}), AIMessage(content='다람쥐가 도토리 나무를 세 그루 키우고 있다니, 흥미로운 이야기네요! 다람쥐들은 도토리를 좋아하고, 때로는 도토리를 땅에 묻어 두었다가 나중에 먹곤 하죠. 혹시 다람쥐와 도토리 나무에 대한 이야기를 더 공유해 주실 건가요? 또는 어떤 특정한 질문이 있으신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 44, 'total_tokens': 144, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CBwc7WYTU5R5fOvGYKt7MtgSdLCdV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c42da767-5196-4d0c-a98a-77e361c9ceb2-0', usage_metadata={'input_tokens': 44, 'output_tokens': 100, 'total_tokens': 144, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_tok